<a href="https://colab.research.google.com/github/sipocz/_earthQuake/blob/main/_earthQuake__PC_colabversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.ensemble import IsolationForest
import numpy as np
from sklearn.metrics import accuracy_score



#EarthQuake
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import sweetviz as sw

#!pip install sweetviz
#import sweetviz as sw
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split


def outlierStatistic(X_train_predict):
    print(X_train_predict)
    maxX=len(X_train_predict)
    outlier=0
    for i in X_train_predict:
        if i==-1:
            outlier+=1
    print(f"A összes ({maxX} darabból {outlier} darab outlier van. Az {outlier/maxX*100:5.1f} %.)")

basedir="C:/Users/sipocz/OneDrive/Dokumentumok/GitHub"
basedir="/content/drive/My Drive/001_AI"

df=pd.read_csv(basedir+"/_EarthQuake/features_a.csv")
df_clasters=pd.read_csv(basedir+"/_EarthQuake/train_labels.csv")
df_testvalues=pd.read_csv(basedir+"/_EarthQuake/test_a.csv")
#official_testvalues=pd.read_csv("/content/drive/My Drive/001_AI/_EarthQuake/test_values.csv")

# Adatfeldolgozás, beolvasások 

numx=260601
#numx=50000
Y=df_clasters[["damage_grade"]]
X=df
scaler=MinMaxScaler()
Xt=X[:numx]
Xt=scaler.fit_transform(Xt)
Xt=Xt[:numx]
Y=Y[["damage_grade"]]
Y=Y[:numx]
X_train, X_test, y_train, y_test = train_test_split(Xt, Y, random_state=2,test_size=0.0001)
df_buildings=df_testvalues[["building_id"]]
heads=df.columns
print(len(heads))
print(len(df_testvalues.columns))
for i in df_testvalues.columns:
    if i not in heads:
        df_testvalues.drop(columns=[i], inplace=True)

print(len(df_testvalues.columns))
print(len(X_train))
print(X_train)

# inlier és outlier bombázás
# -------------------- Az ötlet -----------------------
# Szedjük a halmazt kétfele 
# inlierekre és outlierekre 
# kezeljük őket külön
# a külön halmazokra illesszünk egy Classifiert 
# OSztályozzuk mindkét halmazt a saját klasszifier modellje szerint
# Fűzzük egybe az adatokat  
#
#

clf = IsolationForest(n_estimators=117, warm_start=False, max_features=45)
clf.fit(X_train)  # fit 10 trees  
#clf.set_params()  # add 10 more trees  
#clf.fit(X)  # fit the added trees 

X_train_predict=clf.predict(X_train)
print(len(X_train))
print(X_train)


outlierStatistic(X_train_predict)

# az official test vizsgálata outlierekre nézve:
outlier_official_predikt=clf.predict(df_testvalues)

outlierStatistic(outlier_official_predikt)

print(list(y_train[["damage_grade"]].index)[0])

print(X_train[1])


y_train["damage_grade"].values


# Szétszedjük a rendszer inlierekre és outlierekre !
X_train_inliers=[X_train[inx] for inx,i in enumerate(X_train_predict) if i==1 ]
X_train_outliers=[X_train[inx] for inx,i in enumerate(X_train_predict) if i==-1 ]
tmp=list(y_train["damage_grade"].values)
y_train_inliers=[tmp[inx] for inx,i in enumerate(X_train_predict) if i==1 ]
y_train_outliers=[tmp[inx] for inx,i in enumerate(X_train_predict) if i==-1 ]



# ---------------- Inlier modell Start ----------------
# konvertáljuk DataFrame - mé a listákat.
dX1=pd.DataFrame (X_train_inliers)
dY1=pd.DataFrame (y_train_inliers)
DtX=pd.DataFrame(X_test)

# KNN inlier modell

from xgboost import XGBClassifier  # 72.09
# max_depth=10 : 72.79857561664441

knn_inlier = XGBClassifier(verbosity=3,max_depth = 35)

print("Fit: Inlier betanitás Start--")
knn_inlier.fit(dX1, dY1)
print("Fit End - Prediction Start ")

y_pred=knn_inlier.predict(DtX)
print("Prediction End")



# Inlier statistics
accuracy =  accuracy_score(y_test, y_pred) * 100
print(f"Accuracy a testhalmazra nézve: {accuracy}")
y_pred2=knn_inlier.predict(dX1)
accuracy =  accuracy_score(y_pred2, y_train_inliers) * 100
print(f"Accuracy a betanított halmazra : {accuracy}")

# ------------- Inlier model vége ---------------

#-------------- Outlier MOdel Start --------------
# DataFrame készítése 
dX1_ol=pd.DataFrame (X_train_outliers)
dY1_ol=pd.DataFrame (y_train_outliers)
DtX=pd.DataFrame(X_test)

# készítsünk modell az outlierekre is
# KNN outlier modell

from xgboost import XGBClassifier  
knn_outlier = XGBClassifier(verbosity=3,max_depth = 35)

print("Fit Start--")
knn_outlier.fit(dX1_ol, dY1_ol)
print("Fit End--")

from sklearn.metrics import accuracy_score
y_pred_ol=knn_outlier.predict(dX1_ol)
accuracy =  accuracy_score(y_pred_ol, y_train_outliers) * 100
print(f"Accuracy a betanított halmazra : {accuracy}")



# megvan a két modell, inlierekre és outlierekre



Mounted at /content/drive
57
78
57
260574
[[0.57266692 0.26666667 0.70707779 ... 0.         0.         1.        ]
 [0.9415848  0.86666667 0.71688858 ... 0.         0.         1.        ]
 [0.12814275 0.13333333 0.41205326 ... 0.         0.         1.        ]
 ...
 [0.77991174 0.86666667 0.51857043 ... 0.         0.         1.        ]
 [0.38710284 0.16666667 0.18710582 ... 0.         0.         1.        ]
 [0.34250192 0.26666667 0.99088998 ... 0.         0.         1.        ]]
260574
[[0.57266692 0.26666667 0.70707779 ... 0.         0.         1.        ]
 [0.9415848  0.86666667 0.71688858 ... 0.         0.         1.        ]
 [0.12814275 0.13333333 0.41205326 ... 0.         0.         1.        ]
 ...
 [0.77991174 0.86666667 0.51857043 ... 0.         0.         1.        ]
 [0.38710284 0.16666667 0.18710582 ... 0.         0.         1.        ]
 [0.34250192 0.26666667 0.99088998 ... 0.         0.         1.        ]]
[1 1 1 ... 1 1 1]
A összes (260574 darabból 28842 darab outlier

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[18:19:41] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14150 extra nodes, 0 pruned nodes, max_depth=35
[18:19:47] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 50910 extra nodes, 0 pruned nodes, max_depth=35
[18:19:53] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 50964 extra nodes, 0 pruned nodes, max_depth=35
[18:19:57] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14660 extra nodes, 0 pruned nodes, max_depth=34
[18:20:03] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 52584 extra nodes, 0 pruned nodes, max_depth=35
[18:20:09] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 52656 extra nodes, 0 pruned nodes, max_depth=35
[18:20:14] INFO: /workspace/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14922 extra nodes, 0 pruned nodes, max_depth=35
[18:20:20] INFO: /workspace/src/tree/updater_prune.cc:74: tree prunin

In [11]:

#az eredeti adatok itt vannak
# ---------------------------------------------------------------------
df_testvalues  #teszteljuk vissza a rendszert  egy eredeti halmazzal 
#----------------------------------------------------------------------
# pl így kapjuk szét az eredeti adatokat
# -- demo adatok esetén --
#X_train, X_test, y_train, y_test = train_test_split(Xt, Y, random_state=2,test_size=0.5)
#df_testvalues=X_test
#----------------------------------------------------------------------
# szét kellene kapni inlierekre és outlierekre a korábban betanított modellel.
test_outliers=clf.predict(df_testvalues)

# nézzünk egy statisztikát
outlierStatistic(test_outliers)

#szedjük szét inlier és outlier listákra

list_testvalues=df_testvalues.values.tolist()
print(list_testvalues[0])
X_test_inliers=[list_testvalues[inx] for inx,i in enumerate(test_outliers) if i==1 ]
X_test_outliers=[list_testvalues[inx] for inx,i in enumerate(test_outliers) if i==-1 ]

df_X_test_inliers=pd.DataFrame (X_test_inliers)
df_X_test_outliers=pd.DataFrame (X_test_outliers)


[-1 -1 -1 ... -1 -1 -1]
A összes (86868 darabból 83700 darab outlier van. Az  96.4 %.)
[0.0, 17.0, 596.0, 3.0, 20.0, 7.0, 6.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0]


In [12]:
df_testvalues.head()

,Unnamed: 0,geo_level_1_id,geo_level_2_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_other,land_surface_condition_t,land_surface_condition_o,foundation_type_h,foundation_type_w,foundation_type_i,foundation_type_r,foundation_type_u,roof_type_q,roof_type_n,roof_type_x,ground_floor_type_z,ground_floor_type_v,ground_floor_type_f,ground_floor_type_m,other_floor_type_q,other_floor_type_s,other_floor_type_j,other_floor_type_x,position_j,position_s,position_t,plan_configuration_c,plan_configuration_s,plan_configuration_d,plan_configuration_a,plan_configuration_u,plan_configuration_o,legal_ownership_status_a,legal_ownership_status_w,legal_ownership_status_v
0,0,17,596,3,20,7,6,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1,6,141,2,25,13,5,0,1,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,2,22,19,2,5,4,5,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
3,3,26,39,1,0,19,3,0,0,0,0,0,1,0,0,0,0,0,2,1,0,0,1,0,0,0,0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,4,17,289,3,15,8,7,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [13]:


#predikáljuk az értékeket
#van betanított inlier predikátorunk
#meg van egy az outlierekre is 

y_inlier_predikt=knn_inlier.predict(df_X_test_inliers)
y_outlier_predikt=knn_outlier.predict(df_X_test_outliers)

df_test_buildings=pd.read_csv(basedir+"/_EarthQuake/test_a.csv")
df_buildings=df_test_buildings[["building_id"]]

building_id=df_buildings[["building_id"]].values

print("building_id hossza:",len(building_id))
print("test_outliers hossza:",len(test_outliers))


building_id hossza: 86868
test_outliers hossza: 86868


In [16]:

building=[]
damage=[]
opi=0 # outlier predikt index
ipi=0 # inlier predikt index
bid=0
#össze kellene rakni az eredeti listát
for idx,i in enumerate(test_outliers):
    building.append(building_id[idx][0])  # idx-el kell hivatkozni 
  # a buildin egyelőre nem érdekel, nem végleges adtokkal foglalkozunk !!!
    if i==-1:
        damage.append(y_outlier_predikt[opi])
        opi+=1
    else:
        damage.append(y_inlier_predikt[ipi])    
        ipi+=1
# -- demo adatok esetén --
#accuracy =  accuracy_score(damage, y_test) * 100

#print("-----------###------------------------")
#print(f"Accuracy a demóra halmazra : {accuracy}")


print(len(building))



outdf=pd.DataFrame(data={"damage_grade":damage} ,index=building)
outdf.index.name="building_id"


outdf.head()

basedir="/content/drive/My Drive/001_AI"
outdf.to_csv(basedir+"/_EarthQuake/submission_7_outlier_jav.csv")

#!head "/content/drive/My Drive/001_AI/_EarthQuake/submission_7_outlier_jav.csv"

#!head "/content/drive/My Drive/001_AI/_EarthQuake/test_a.csv"




86868


In [19]:
!head "/content/drive/My Drive/001_AI/_EarthQuake/submission_7_outlier_jav.csv"

building_id,damage_grade
300051,3
99355,3
890251,3
745817,2
421793,3
871976,2
691228,2
896100,3
343471,2
